In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [3]:
# Load the dataset
df = pd.read_csv('..//..//dataset//final_dataset.csv')  # Replace with your dataset path
# df=pd.read_excel('..//..//dataset//Dataset_IoV.xlsx')

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']
# scaler=RobustScaler()
# X_scaled = scaler.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [4]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(320000, 1, 6) (320000,) (400000, 1, 6)


# Model

In [5]:
def build_deep_rnn_bi_lstm():
    model = Sequential()
    
    # Add multiple SimpleRNN layers for Deep RNN
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(SimpleRNN(units=50, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(SimpleRNN(units=50, return_sequences=True))
    model.add(Dropout(0.1))

    # Add multiple Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=50)))
    model.add(Dropout(0.1))

    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])

    return model


In [6]:
# Early stopping callback
early_stopping = EarlyStopping(
    monitor='mse',  
    patience=3,   
    restore_best_weights=True, 
    verbose=1
)
# Model checkpoint callback
best_weights_filepath ='..//models//best_model.keras'

model_checkpoint = ModelCheckpoint(
    best_weights_filepath,  
    monitor='val_mse',  
    mode='min',       
    save_best_only=True,   
    verbose=1
)

# Instantiate and train the model
model = build_deep_rnn_bi_lstm()
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2, verbose=1,   
#  callbacks=[model_checkpoint,early_stopping]
)

Epoch 1/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.0563 - mae: 0.0422 - mse: 0.0184 - val_loss: 3.3755e-06 - val_mae: 3.3740e-06 - val_mse: 1.2233e-09
Epoch 2/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - loss: 1.0209e-04 - mae: 4.5075e-05 - mse: 2.0517e-05 - val_loss: 2.8452e-06 - val_mae: 2.8503e-06 - val_mse: 1.3472e-11
Epoch 3/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - loss: 6.4216e-05 - mae: 2.7884e-05 - mse: 1.0143e-05 - val_loss: 1.4317e-06 - val_mae: 1.4302e-06 - val_mse: 2.6255e-12
Epoch 4/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - loss: 5.3791e-05 - mae: 7.1612e-06 - mse: 4.0063e-06 - val_loss: 8.3147e-07 - val_mae: 8.2957e-07 - val_mse: 9.3912e-13
Epoch 5/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - loss: 5.8839e-05 - mae: 2.0847e-05 - mse: 1.3150e-05 - val_loss: 2.6966e-06 - val_mae: 2.6987e-06 - val_mse: 8.8987e-12


In [7]:
# import numpy as np

# # Set numpy to display full float numbers
# np.set_printoptions(suppress=True, precision=10)

In [8]:
# results = model.evaluate(X_test, y_test, verbose=1)
# loss, mse, mae = results  # Unpack all the returned metrics
# print(f'Loss: {loss:.4f}')
# print(f'Mean Squared Error (MSE): {mse:.4f}')
# print(f'Mean Absolute Error (MAE): {mae:.4f}')


In [9]:

from sklearn.metrics import mean_squared_error
import numpy as np

# Make predictions
y_pred = model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step
Mean Squared Error (MSE): 0.0000000000088626
Root Mean Squared Error (RMSE): 0.0000029770105952


In [10]:
print(y_pred)
y_pred=y_pred.flatten()
print(y_pred)

[[1.4188697e-06]
 [9.9999553e-01]
 [1.4188697e-06]
 ...
 [9.9999553e-01]
 [1.3827974e-06]
 [9.9999613e-01]]
[1.4188697e-06 9.9999553e-01 1.4188697e-06 ... 9.9999553e-01 1.3827974e-06
 9.9999613e-01]


In [11]:
# accuracy
y_pred = np.round(y_pred)
accuracy = np.mean(y_test == y_pred)
print(f'Accuracy: {accuracy:.16f}')

Accuracy: 1.0000000000000000


In [12]:
ones_count_actual = np.count_nonzero(y_test)
zeros_count_actual = len(y_test) - ones_count_actual

ones_count_predicted = np.count_nonzero(y_pred)
zeros_count_predicted = len(y_pred) - ones_count_predicted

print(f'Actual: ones={ones_count_actual}, zeros={zeros_count_actual}')
print(f'Predicted: ones={ones_count_predicted}, zeros={zeros_count_predicted}')

Actual: ones=39866, zeros=40134
Predicted: ones=39866, zeros=40134


In [13]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()


In [14]:
#saving the model
# model.save('..//models//deep_rnn_bi_lstm_updated_1.keras')  # Replace with your model path